In [14]:
import shutil                          
import glob                             
import numpy as np
import cv2
import matplotlib.pyplot as plt
%matplotlib inline
import os
import pandas as pd
import sklearn
from IPython.display import display

In [15]:
from RNASeq_prep_n_RFOptimization import *

In [19]:
os.getcwd()                             
root_path = "#########################"
met_path = root_path + "/genomic_blca/genomic_blca_met"
met_content = next(os.walk(met_path))[1]   
try:
    met_content.remove(".ipynb_checkpoints")  
except:
    pass
#print(met_content)
print(len(met_content))

In [4]:
# Create a list containing path to all "metastasis" rna_seq files.
met_list = []
for barc_folder in met_content:
    barc_folder_path = met_path + "/" + barc_folder
    barc_folder_content = next(os.walk(barc_folder_path))[1]
    met_folder = barc_folder_content[0]
    met_folder_path = barc_folder_path + "/" + met_folder
    met_extension = "/*.tsv"
    rna_seq_files = glob.glob(met_folder_path + met_extension)   
    met_list.append(rna_seq_files[0])
    
#print(met_list)
print(len(met_list))

80


In [5]:
no_met_path = root_path + "/genomic_blca/genomic_blca_no-met"
no_met_content = next(os.walk(no_met_path))[1]  
try:
    no_met_content.remove(".ipynb_checkpoints")  
except:
    pass
#print(no_met_content)
print(len(no_met_content))

80


In [6]:
# Create a list containing path to all "no-metastasis" rna_seq files.
no_met_list = []
for barc_folder in no_met_content:
    barc_folder_path = no_met_path + "/" + barc_folder
    barc_folder_content = next(os.walk(barc_folder_path))[1]
    no_met_folder = barc_folder_content[0]
    no_met_folder_path = barc_folder_path + "/" + no_met_folder
    no_met_extension = "/*.tsv"
    rna_seq_files = glob.glob(no_met_folder_path + no_met_extension)  
    no_met_list.append(rna_seq_files[0])
    
#print(no_met_list)
print(len(no_met_list))

80


### Get raw count dataframe

In [7]:
met_unstranded_objs = all_tpm_objs_list(met_list, data_type = "unstranded", metastasis=True)
met_objs = met_unstranded_objs[0]
met_labels = met_unstranded_objs[1]

In [8]:
no_met_unstranded_objs = all_tpm_objs_list(no_met_list, data_type = "unstranded", metastasis=False)
no_met_objs = no_met_unstranded_objs[0]
no_met_labels = no_met_unstranded_objs[1]

In [11]:
all_cat_unstranded_objs = met_objs + no_met_objs

In [13]:
# Merge each rna-seq df into one full df
all_cat_full_df = merge_dataframe(all_cat_unstranded_objs)
all_cat_full_df = all_cat_full_df.set_index(["gene_name"])
all_cat_full_df

,met_patient1,met_patient2,met_patient3,met_patient4,met_patient5,met_patient6,met_patient7,met_patient8,met_patient9,met_patient10,...,patient71,patient72,patient73,patient74,patient75,patient76,patient77,patient78,patient79,patient80
gene_name,,,,,,,,,,,,,,,,,,,,,
TSPAN6,7301,3828,2454,8697,1480,4225,11615,4368,10148,6320,...,1448,2238,4754,2053,3314,1705,4110,1791,8875,12373
TNMD,0,0,0,0,3,0,4,1,3,0,...,4,1,1,1,0,338,0,0,0,0
DPM1,3079,3100,2575,1574,2471,2577,2649,2053,1482,1746,...,1515,2743,2851,332,2057,608,1637,388,2610,1795
SCYL3,1098,405,220,608,519,786,821,872,959,1314,...,1091,722,1074,359,1094,312,434,204,1433,3725
C1orf112,561,480,312,282,613,899,819,334,623,318,...,709,503,1087,116,1044,489,571,61,323,1829
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
AL451106.1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC008763.4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
AC006486.3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [14]:
# Save raw count
all_cat_full_df.to_csv("deseq2_rawcount_blca.csv")

### Get samples information

In [16]:
# Conditions (for DESeq2 Samples data dataframe).. basically like label
met_lst = ["met"] * 80
no_met_lst = ["no_met"] * 80
cond_lst = met_lst + no_met_lst

In [17]:
# Sample names
samples_name = list(all_cat_full_df.columns)

In [18]:
sample_info = pd.DataFrame({"samples": samples_name, "conditions": cond_lst})
sample_info = sample_info.set_index(["samples"])
#sample_info

In [19]:
sample_info.to_csv("deseq2_sample_info_blca.csv")